In [1]:
import pandas as pd
import gc


In [2]:
user_log = pd.read_csv(r'C:\Users\Superstar\Desktop\data_format1\data_format1\user_log_format1.csv').drop_duplicates()
user_log = user_log.dropna(axis=0,how='all')
user_log.head()

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
0,328862,323294,833,2882,2661.0,829,0
1,328862,844400,1271,2882,2661.0,829,0
2,328862,575153,1271,2882,2661.0,829,0
3,328862,996875,1271,2882,2661.0,829,0
4,328862,1086186,1271,1253,1049.0,829,0


In [3]:
user_log.isnull().sum()# 检查缺失值

user_id            0
item_id            0
cat_id             0
seller_id          0
brand_id       87405
time_stamp         0
action_type        0
dtype: int64

In [4]:
user_log.brand_id.fillna(user_log.brand_id.mode()[0],inplace=True)# 品牌用众数填充

In [5]:
predict = pd.read_csv(r"C:\Users\Superstar\Desktop\sample_submission.csv")
predict.rename(columns={'merchant_id':'seller_id'},inplace = True)
predict.head()


,user_id,seller_id,prob
0,163968,4605,0.5
1,360576,1581,0.5
2,98688,1964,0.5
3,98688,3645,0.5
4,295296,3361,0.5


In [6]:
test = pd.merge(user_log,predict,how='left',on=['user_id','seller_id'])
test.head()


,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,prob
0,328862,323294,833,2882,2661.0,829,0,NaN
1,328862,844400,1271,2882,2661.0,829,0,NaN
2,328862,575153,1271,2882,2661.0,829,0,NaN
3,328862,996875,1271,2882,2661.0,829,0,NaN
4,328862,1086186,1271,1253,1049.0,829,0,NaN


###################################新加入

In [7]:
test.dropna(axis=0,how='any',inplace=True)
del test['prob']
test.head()

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
147,328862,406349,1280,2700,5476.0,1111,0
149,328862,406349,1280,2700,5476.0,1111,2
497,272389,1112375,1112,4186,6035.0,1111,0
498,272389,1070749,1112,4186,6035.0,1111,0
499,272389,1112375,1112,4186,6035.0,1111,2


In [8]:
test['train_test']='test'

In [9]:
test.shape

(1792412, 8)

########################################

In [11]:
user_info = pd.read_csv(r'C:\Users\Superstar\Desktop\data_format1\data_format1\user_info_format1.csv')
user_info = user_info.dropna(axis=0,how='all')
user_info.head()
print(user_info.shape)

(424170, 3)


In [12]:
user_info.isnull().sum()#检查缺失值

user_id         0
age_range    2217
gender       6436
dtype: int64

In [13]:
user_info.gender.fillna(user_info.gender.mode()[0],inplace=True)# 性别用众数填充
user_info.age_range.fillna(user_info.age_range.median(),inplace=True)# 年龄用中位数填充

In [14]:
# train = pd.read_csv(r'C:\Users\Superstar\Desktop\data_format1\data_format1\train_format1.csv')
# train = train.drop_duplicates(['user_id','merchant_id'])
# train.head()

In [15]:
# test = pd.read_csv(r'C:\Users\Superstar\Desktop\data_format1\data_format1\test_format1.csv')
# test = test.drop_duplicates(['user_id','merchant_id'])
# test.head()

In [16]:
eta = 0.1 #取样比例
user_log = user_log.sample(int(len(user_log)*eta))
user_log['train_test']='train'
user_log = pd.concat([user_log,test],axis=0)

In [17]:
user_log = user_log.merge(user_info, on=['user_id'],how='left',copy=False)

In [18]:
user_log.shape

(5909925, 10)

In [19]:

# user_log['month'] = user_log['time_stamp']//100
# user_log['day'] = user_log['time_stamp']%100
#--------------zyy -----------------

#--------------zyy -----------------
print(user_log.shape[0])
user_log.head()

5909925


,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,train_test,age_range,gender
0,66912,203092,1095,3125,7118.0,1110,0,train,5.0,0.0
1,66632,461173,737,4564,1803.0,1103,0,train,3.0,1.0
2,122117,1018773,737,82,2971.0,807,0,train,0.0,0.0
3,173592,383058,1113,1358,966.0,604,0,train,3.0,0.0
4,381740,927745,302,3546,3939.0,1013,0,train,0.0,0.0


In [20]:
df_age = pd.get_dummies(user_info.age_range,prefix='age')# 对age进行哑编码
df_gender = pd.get_dummies(user_info.gender)# 对gender进行哑编码
df_gender.rename(columns={0:'female',1:'male',2:'unknown'},inplace=True)
user_info = pd.concat([user_info.user_id, df_age, df_gender], axis=1)
user_info.head()

,user_id,age_0.0,age_1.0,age_2.0,age_3.0,age_4.0,age_5.0,age_6.0,age_7.0,age_8.0,female,male,unknown
0,376517,0,0,0,0,0,0,1,0,0,0,1,0
1,234512,0,0,0,0,0,1,0,0,0,1,0,0
2,344532,0,0,0,0,0,1,0,0,0,1,0,0
3,186135,0,0,0,0,0,1,0,0,0,1,0,0
4,30230,0,0,0,0,0,1,0,0,0,1,0,0


In [21]:
user_log.shape

(5909925, 10)

In [22]:
#采用onehot编码统计用户交互情况
total_action = user_log[["user_id","action_type"]]
action = pd.get_dummies(total_action['action_type'],prefix='user_action')
action.head()

,user_action_0,user_action_1,user_action_2,user_action_3
0,1,0,0,0
1,1,0,0,0
2,1,0,0,0
3,1,0,0,0
4,1,0,0,0


In [23]:
#统计用户点击、加入购物车、购买、加入收藏夹次数
total_action = pd.concat([total_action.user_id,action], axis=1).groupby(['user_id'], as_index=False).sum()

total_action.head()


,user_id,user_action_0,user_action_1,user_action_2,user_action_3
0,1,2.0,0.0,0.0,0.0
1,2,6.0,0.0,3.0,1.0
2,3,5.0,0.0,2.0,2.0
3,4,3.0,0.0,0.0,0.0
4,5,12.0,0.0,2.0,3.0


In [24]:
#总次数
total_action['total_action'] = total_action['user_action_0']+total_action['user_action_1']+total_action['user_action_2']+total_action['user_action_3']
total_action.head()


,user_id,user_action_0,user_action_1,user_action_2,user_action_3,total_action
0,1,2.0,0.0,0.0,0.0,2.0
1,2,6.0,0.0,3.0,1.0,10.0
2,3,5.0,0.0,2.0,2.0,9.0
3,4,3.0,0.0,0.0,0.0,3.0
4,5,12.0,0.0,2.0,3.0,17.0


In [25]:
total_action.shape[0]

412614

In [26]:
user_info = pd.merge(user_log,total_action,how='left',on=['user_id'])
#-------------------zyy
del action
gc.collect()
#-------------------zyy
user_info.head()

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,train_test,age_range,gender,user_action_0,user_action_1,user_action_2,user_action_3,total_action
0,66912,203092,1095,3125,7118.0,1110,0,train,5.0,0.0,11.0,0.0,1.0,1.0,13.0
1,66632,461173,737,4564,1803.0,1103,0,train,3.0,1.0,18.0,0.0,2.0,0.0,20.0
2,122117,1018773,737,82,2971.0,807,0,train,0.0,0.0,9.0,0.0,1.0,2.0,12.0
3,173592,383058,1113,1358,966.0,604,0,train,3.0,0.0,45.0,0.0,5.0,22.0,72.0
4,381740,927745,302,3546,3939.0,1013,0,train,0.0,0.0,16.0,0.0,2.0,0.0,18.0


In [28]:
user_info.shape

(5909925, 15)

In [31]:
item_num = user_log.groupby(['user_id'])['item_id'].nunique()#商品数量
cat_num = user_log.groupby(['user_id'])['cat_id'].nunique()#交互次数
seller_num = user_log.groupby(['user_id'])['seller_id'].nunique()#商店数量
brand_num = user_log.groupby(['user_id'])['brand_id'].nunique()#品牌数量
time_num = user_log.groupby(['user_id'])['time_stamp'].nunique()#做出交互的天数
info = pd.concat([item_num,cat_num,seller_num,brand_num,time_num],axis=1)
info.rename(columns={'item_id':'item_num','cat_id':'cat_num','seller_id':'seller_num','brand_id':'brand_num','time_stamp':'time_num'},inplace=True)
del item_num,cat_num,seller_num,brand_num,time_num
#-----------------------zyy
gc.collect()
#-----------------------zyy
info.head()

,item_num,cat_num,seller_num,brand_num,time_num
user_id,,,,,
1,2,1,2,2,1
2,10,7,6,6,4
3,5,5,3,3,3
4,3,3,3,3,3
5,15,11,11,12,10


In [32]:
info.reset_index(inplace=True)
info.head()

,user_id,item_num,cat_num,seller_num,brand_num,time_num
0,1,2,1,2,2,1
1,2,10,7,6,6,4
2,3,5,5,3,3,3
3,4,3,3,3,3,3
4,5,15,11,11,12,10


In [33]:
info.shape

(412614, 6)

In [34]:
user_info = pd.merge(user_info,info,how='left',on=['user_id'])
del info
#--------------------zyy
gc.collect()
#--------------------zyy
user_info.head()

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,train_test,age_range,gender,user_action_0,user_action_1,user_action_2,user_action_3,total_action,item_num,cat_num,seller_num,brand_num,time_num
0,66912,203092,1095,3125,7118.0,1110,0,train,5.0,0.0,11.0,0.0,1.0,1.0,13.0,13,8,4,4,3
1,66632,461173,737,4564,1803.0,1103,0,train,3.0,1.0,18.0,0.0,2.0,0.0,20.0,18,12,11,11,5
2,122117,1018773,737,82,2971.0,807,0,train,0.0,0.0,9.0,0.0,1.0,2.0,12.0,11,7,10,10,6
3,173592,383058,1113,1358,966.0,604,0,train,3.0,0.0,45.0,0.0,5.0,22.0,72.0,70,35,50,52,43
4,381740,927745,302,3546,3939.0,1013,0,train,0.0,0.0,16.0,0.0,2.0,0.0,18.0,13,10,11,11,12


In [36]:
user_info.shape

(5909925, 20)

In [37]:
user_info.isnull().sum()

user_id          0
item_id          0
cat_id           0
seller_id        0
brand_id         0
time_stamp       0
action_type      0
train_test       0
age_range        0
gender           0
user_action_0    0
user_action_1    0
user_action_2    0
user_action_3    0
total_action     0
item_num         0
cat_num          0
seller_num       0
brand_num        0
time_num         0
dtype: int64

In [38]:
#创建seller特征的dataframe
seller_info = user_log[['seller_id']]
seller_info.head()

,seller_id
0,3125
1,4564
2,82
3,1358
4,3546


In [39]:
#提取商店特征
seller_action = pd.get_dummies(user_log['action_type'],prefix='seller_action')
seller_action = pd.concat([user_log.seller_id, seller_action], axis=1).groupby(['seller_id'], as_index=False).sum()
seller_action.head()


,seller_id,seller_action_0,seller_action_1,seller_action_2,seller_action_3
0,1,22480.0,38.0,1601.0,1320.0
1,2,233.0,2.0,74.0,19.0
2,3,191.0,1.0,6.0,20.0
3,4,199.0,1.0,28.0,13.0
4,5,564.0,0.0,13.0,58.0


In [40]:
#总次数
seller_action['total_action_seller'] = seller_action.sum(axis = 1)
seller_action.head()

,seller_id,seller_action_0,seller_action_1,seller_action_2,seller_action_3,total_action_seller
0,1,22480.0,38.0,1601.0,1320.0,25440.0
1,2,233.0,2.0,74.0,19.0,330.0
2,3,191.0,1.0,6.0,20.0,221.0
3,4,199.0,1.0,28.0,13.0,245.0
4,5,564.0,0.0,13.0,58.0,640.0


In [41]:
seller_info.shape

(5909925, 1)

In [42]:
seller_info = pd.merge(seller_info,seller_action,how='left',on=['seller_id'])
seller_info.head()
#------------------zyy
del seller_action
gc.collect()
#------------------zyy

40

In [43]:
seller_info.isnull().sum()

seller_id              0
seller_action_0        0
seller_action_1        0
seller_action_2        0
seller_action_3        0
total_action_seller    0
dtype: int64

In [44]:
seller_info.shape

(5909925, 6)

In [45]:
item_num = user_log.groupby(['seller_id'])['item_id'].nunique()#商品数量
cat_num = user_log.groupby(['seller_id'])['cat_id'].nunique()#交互次数
brand_num = user_log.groupby(['seller_id'])['brand_id'].nunique()#品牌数量
time_num = user_log.groupby(['seller_id'])['time_stamp'].nunique()#做出交互的天数
info = pd.concat([item_num,cat_num,brand_num,time_num],axis=1)
info.rename(columns={'item_id':'item_num_seller','cat_id':'cat_num_seller','brand_id':'brand_num_seller','time_stamp':'time_num_seller'},inplace=True)
info.reset_index(inplace = True)
#------------------zyy
print(info.head())
del item_num, cat_num, brand_num, time_num
gc.collect()
#------------------zyy

   seller_id  item_num_seller  cat_num_seller  brand_num_seller  \
0          1             2306              35                 3   
1          2               73               8                 2   
2          3               62               4                 2   
3          4               64               5                 3   
4          5              205              18                 1   

   time_num_seller  
0              185  
1               77  
2               83  
3              119  
4              159  


0

In [46]:
seller_info = pd.merge(seller_info,info,how='left',on=['seller_id'])
del info
#-------------------zyy
gc.collect()
#-------------------zyy
seller_info.head()


(5909925, 10)


In [47]:
seller_info.isnull().sum()

seller_id              0
seller_action_0        0
seller_action_1        0
seller_action_2        0
seller_action_3        0
total_action_seller    0
item_num_seller        0
cat_num_seller         0
brand_num_seller       0
time_num_seller        0
dtype: int64

In [48]:
#提取用户-商家特征
user_seller_log = pd.read_csv(r'C:\Users\Superstar\Desktop\data_format1\data_format1\train_format1.csv')
#-------------------zyy
# user_seller_log = user_seller_log.sample(int(len(user_seller_log)*eta))
#-------------------zyy
user_seller_log.head()

,user_id,merchant_id,label
0,34176,3906,0
1,34176,121,0
2,34176,4356,1
3,34176,2217,0
4,230784,4818,0


In [49]:
user_seller_log.shape

(260864, 3)

In [50]:
user_seller_log.rename(columns={'merchant_id':'seller_id'},inplace = True)
user_seller_log = pd.merge(user_seller_log, user_log, on=['seller_id','user_id'])
user_seller_log.head()

,user_id,seller_id,label,item_id,cat_id,brand_id,time_stamp,action_type,train_test,age_range,gender
0,34176,3906,0,320263,662,6268.0,1027,0,train,6.0,0.0
1,34176,3906,0,757713,821,6268.0,1111,0,train,6.0,0.0
2,34176,121,0,800752,1028,2337.0,1108,0,train,6.0,0.0
3,34176,4356,1,709909,1208,2276.0,1110,0,train,6.0,0.0
4,34176,4356,1,709909,1208,2276.0,1111,2,train,6.0,0.0


In [51]:
user_seller_info = user_seller_log[['user_id','seller_id']]
action = pd.get_dummies(user_seller_log['action_type'],prefix='user_seller_action')
user_seller_info = pd.concat([user_seller_info, action], axis=1).groupby(['user_id', 'seller_id'], as_index=False).sum()
#-----------------zyy
# 此处不用清除数据
#-----------------zyy
user_seller_info.head()

,user_id,seller_id,user_seller_action_0,user_seller_action_1,user_seller_action_2,user_seller_action_3
0,6,4249,1,0,0,0
1,9,2721,1,0,1,0
2,14,361,2,0,0,1
3,16,1435,1,0,0,0
4,18,4079,1,0,0,0


In [52]:
user_seller_info['total_action'] = user_seller_info['user_seller_action_0'] + user_seller_info['user_seller_action_1'] + user_seller_info['user_seller_action_2'] + user_seller_info['user_seller_action_3']
user_seller_info.head()

,user_id,seller_id,user_seller_action_0,user_seller_action_1,user_seller_action_2,user_seller_action_3,total_action
0,6,4249,1,0,0,0,1
1,9,2721,1,0,1,0,2
2,14,361,2,0,0,1,3
3,16,1435,1,0,0,0,1
4,18,4079,1,0,0,0,1


In [53]:
user_seller_info.shape

(101185, 7)

In [54]:
# 点击量
num = user_seller_log[user_seller_log.action_type == 0]
item_num = num.groupby(['user_id', 'seller_id']).apply(lambda x:x.item_id.nunique()).reset_index()
cat_num = num.groupby(['user_id', 'seller_id']).apply(lambda x:x.cat_id.nunique()).reset_index()
brand_num = num.groupby(['user_id', 'seller_id']).apply(lambda x:x.brand_id.nunique()).reset_index()
item_num.rename(columns={0:'item_number_0'},inplace=True)
cat_num.rename(columns={0:'cat_number_0'},inplace=True)
brand_num.rename(columns={0:'brand_number_0'},inplace=True)
user_seller_info = pd.merge(user_seller_info,item_num,how='left',on=['user_id', 'seller_id'])
user_seller_info = pd.merge(user_seller_info,cat_num,how='left',on=['user_id','seller_id'])
user_seller_info = pd.merge(user_seller_info,brand_num,how='left',on=['user_id','seller_id'])
user_seller_info.item_number_0.fillna(0,inplace = True)
user_seller_info.cat_number_0.fillna(0,inplace = True)
user_seller_info.brand_number_0.fillna(0,inplace = True)
del num,item_num,cat_num,brand_num
gc.collect()
user_seller_info.head()

,user_id,seller_id,user_seller_action_0,user_seller_action_1,user_seller_action_2,user_seller_action_3,total_action,item_number_0,cat_number_0,brand_number_0
0,6,4249,1,0,0,0,1,1.0,1.0,1.0
1,9,2721,1,0,1,0,2,1.0,1.0,1.0
2,14,361,2,0,0,1,3,2.0,1.0,1.0
3,16,1435,1,0,0,0,1,1.0,1.0,1.0
4,18,4079,1,0,0,0,1,1.0,1.0,1.0


In [55]:
# 加入购物车的数量
num = user_seller_log[user_seller_log.action_type == 1]
item_num = num.groupby(['user_id', 'seller_id']).apply(lambda x:x.item_id.nunique()).reset_index()
cat_num = num.groupby(['user_id', 'seller_id']).apply(lambda x:x.cat_id.nunique()).reset_index()
brand_num = num.groupby(['user_id', 'seller_id']).apply(lambda x:x.brand_id.nunique()).reset_index()
item_num.rename(columns={0:'item_number_1'},inplace=True)
cat_num.rename(columns={0:'cat_number_1'},inplace=True)
brand_num.rename(columns={0:'brand_number_1'},inplace=True)
user_seller_info = pd.merge(user_seller_info,item_num,how='left',on=['user_id', 'seller_id'])
user_seller_info = pd.merge(user_seller_info,cat_num,how='left',on=['user_id','seller_id'])
user_seller_info = pd.merge(user_seller_info,brand_num,how='left',on=['user_id','seller_id'])
user_seller_info.item_number_1.fillna(0,inplace = True)
user_seller_info.cat_number_1.fillna(0,inplace = True)
user_seller_info.brand_number_1.fillna(0,inplace = True)
del num,item_num,cat_num,brand_num
gc.collect()
user_seller_info.head()

,user_id,seller_id,user_seller_action_0,user_seller_action_1,user_seller_action_2,user_seller_action_3,total_action,item_number_0,cat_number_0,brand_number_0,item_number_1,cat_number_1,brand_number_1
0,6,4249,1,0,0,0,1,1.0,1.0,1.0,0.0,0.0,0.0
1,9,2721,1,0,1,0,2,1.0,1.0,1.0,0.0,0.0,0.0
2,14,361,2,0,0,1,3,2.0,1.0,1.0,0.0,0.0,0.0
3,16,1435,1,0,0,0,1,1.0,1.0,1.0,0.0,0.0,0.0
4,18,4079,1,0,0,0,1,1.0,1.0,1.0,0.0,0.0,0.0


In [56]:
# 购买量
num = user_seller_log[user_seller_log.action_type == 2]
item_num = num.groupby(['user_id', 'seller_id']).apply(lambda x:x.item_id.nunique()).reset_index()
cat_num = num.groupby(['user_id', 'seller_id']).apply(lambda x:x.cat_id.nunique()).reset_index()
brand_num = num.groupby(['user_id', 'seller_id']).apply(lambda x:x.brand_id.nunique()).reset_index()
item_num.rename(columns={0:'item_number_2'},inplace=True)
cat_num.rename(columns={0:'cat_number_2'},inplace=True)
brand_num.rename(columns={0:'brand_number_2'},inplace=True)
user_seller_info = pd.merge(user_seller_info,item_num,how='left',on=['user_id', 'seller_id'])
user_seller_info = pd.merge(user_seller_info,cat_num,how='left',on=['user_id','seller_id'])
user_seller_info = pd.merge(user_seller_info,brand_num,how='left',on=['user_id','seller_id'])
user_seller_info.item_number_2.fillna(0,inplace = True)
user_seller_info.cat_number_2.fillna(0,inplace = True)
user_seller_info.brand_number_2.fillna(0,inplace = True)
del num,item_num,cat_num,brand_num
gc.collect()
user_seller_info.head()

,user_id,seller_id,user_seller_action_0,user_seller_action_1,user_seller_action_2,user_seller_action_3,total_action,item_number_0,cat_number_0,brand_number_0,item_number_1,cat_number_1,brand_number_1,item_number_2,cat_number_2,brand_number_2
0,6,4249,1,0,0,0,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9,2721,1,0,1,0,2,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0
2,14,361,2,0,0,1,3,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,16,1435,1,0,0,0,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,18,4079,1,0,0,0,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
# 收藏量
num = user_seller_log[user_seller_log.action_type == 3]
item_num = num.groupby(['user_id', 'seller_id']).apply(lambda x:x.item_id.nunique()).reset_index()
cat_num = num.groupby(['user_id', 'seller_id']).apply(lambda x:x.cat_id.nunique()).reset_index()
brand_num = num.groupby(['user_id', 'seller_id']).apply(lambda x:x.brand_id.nunique()).reset_index()
item_num.rename(columns={0:'item_number_3'},inplace=True)
cat_num.rename(columns={0:'cat_number_3'},inplace=True)
brand_num.rename(columns={0:'brand_number_3'},inplace=True)
user_seller_info = pd.merge(user_seller_info,item_num,how='left',on=['user_id', 'seller_id'])
user_seller_info = pd.merge(user_seller_info,cat_num,how='left',on=['user_id','seller_id'])
user_seller_info = pd.merge(user_seller_info,brand_num,how='left',on=['user_id','seller_id'])
user_seller_info.item_number_3.fillna(0,inplace = True)
user_seller_info.cat_number_3.fillna(0,inplace = True)
user_seller_info.brand_number_3.fillna(0,inplace = True)
del num,item_num,cat_num,brand_num
gc.collect()
user_seller_info.head()

,user_id,seller_id,user_seller_action_0,user_seller_action_1,user_seller_action_2,user_seller_action_3,total_action,item_number_0,cat_number_0,brand_number_0,item_number_1,cat_number_1,brand_number_1,item_number_2,cat_number_2,brand_number_2,item_number_3,cat_number_3,brand_number_3
0,6,4249,1,0,0,0,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9,2721,1,0,1,0,2,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
2,14,361,2,0,0,1,3,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
3,16,1435,1,0,0,0,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,18,4079,1,0,0,0,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
user_seller_info=user_seller_info.astype('int32')
user_seller_info.head()

,user_id,seller_id,user_seller_action_0,user_seller_action_1,user_seller_action_2,user_seller_action_3,total_action,item_number_0,cat_number_0,brand_number_0,item_number_1,cat_number_1,brand_number_1,item_number_2,cat_number_2,brand_number_2,item_number_3,cat_number_3,brand_number_3
0,6,4249,1,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0
1,9,2721,1,0,1,0,2,1,1,1,0,0,0,1,1,1,0,0,0
2,14,361,2,0,0,1,3,2,1,1,0,0,0,0,0,0,1,1,1
3,16,1435,1,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0
4,18,4079,1,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0


In [64]:
user_info

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,train_test,age_range,gender,user_action_0,user_action_1,user_action_2,user_action_3,total_action,item_num,cat_num,seller_num,brand_num,time_num
0,66912,203092,1095,3125,7118.0,1110,0,train,5.0,0.0,11.0,0.0,1.0,1.0,13.0,13,8,4,4,3
1,66632,461173,737,4564,1803.0,1103,0,train,3.0,1.0,18.0,0.0,2.0,0.0,20.0,18,12,11,11,5
2,122117,1018773,737,82,2971.0,807,0,train,0.0,0.0,9.0,0.0,1.0,2.0,12.0,11,7,10,10,6
3,173592,383058,1113,1358,966.0,604,0,train,3.0,0.0,45.0,0.0,5.0,22.0,72.0,70,35,50,52,43
4,381740,927745,302,3546,3939.0,1013,0,train,0.0,0.0,16.0,0.0,2.0,0.0,18.0,13,10,11,11,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5909920,128038,449243,664,2537,6066.0,1111,0,test,2.0,0.0,11.0,0.0,3.0,1.0,15.0,7,4,3,3,2
5909921,128038,84914,664,2537,6066.0,1111,0,test,2.0,0.0,11.0,0.0,3.0,1.0,15.0,7,4,3,3,2
5909922,128038,449243,664,2537,6066.0,1111,2,test,2.0,0.0,11.0,0.0,3.0,1.0,15.0,7,4,3,3,2
5909923,128038,1022583,883,2031,2446.0,1111,2,test,2.0,0.0,11.0,0.0,3.0,1.0,15.0,7,4,3,3,2


In [65]:
# data = pd.merge(user_log,user_info,how='left',on=['user_id'])
# data.head()
data = user_info

In [66]:
del user_info,user_log # 释放一点内存。
gc.collect()

40

In [67]:
seller_info=seller_info.astype('int32')

In [68]:
seller_info.head(10)

,seller_id,seller_action_0,seller_action_1,seller_action_2,seller_action_3,total_action_seller,item_num_seller,cat_num_seller,brand_num_seller,time_num_seller
0,3125,1150,1,23,44,4343,502,42,2,135
1,4564,436,1,99,57,5157,53,3,2,147
2,82,531,0,25,45,683,100,5,1,152
3,1358,297,0,68,46,1769,50,15,2,114
4,3546,329,1,52,28,3956,86,10,2,111
5,1379,789,3,315,43,2529,256,59,19,154
6,3877,1477,2,82,104,5542,218,5,2,179
7,1967,4512,2,116,220,6817,1309,45,13,162
8,1941,363,0,26,21,2351,66,12,1,124
9,2693,729,1,98,53,3574,76,10,2,126


In [69]:
data.head(10)

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,train_test,age_range,gender,user_action_0,user_action_1,user_action_2,user_action_3,total_action,item_num,cat_num,seller_num,brand_num,time_num
0,66912,203092,1095,3125,7118.0,1110,0,train,5.0,0.0,11.0,0.0,1.0,1.0,13.0,13,8,4,4,3
1,66632,461173,737,4564,1803.0,1103,0,train,3.0,1.0,18.0,0.0,2.0,0.0,20.0,18,12,11,11,5
2,122117,1018773,737,82,2971.0,807,0,train,0.0,0.0,9.0,0.0,1.0,2.0,12.0,11,7,10,10,6
3,173592,383058,1113,1358,966.0,604,0,train,3.0,0.0,45.0,0.0,5.0,22.0,72.0,70,35,50,52,43
4,381740,927745,302,3546,3939.0,1013,0,train,0.0,0.0,16.0,0.0,2.0,0.0,18.0,13,10,11,11,12
5,259489,69252,159,1379,7528.0,1111,2,train,4.0,0.0,26.0,1.0,1.0,1.0,29.0,29,18,15,16,16
6,125815,892966,1238,3877,3213.0,1030,0,train,6.0,0.0,22.0,0.0,1.0,0.0,23.0,21,7,10,10,15
7,94709,1104951,1095,1967,5214.0,1105,0,train,5.0,0.0,23.0,0.0,2.0,1.0,26.0,25,19,20,21,16
8,12090,895411,351,1941,4239.0,723,0,train,3.0,0.0,56.0,0.0,2.0,1.0,59.0,57,31,32,35,23
9,303604,974683,821,2693,6581.0,901,0,train,4.0,1.0,15.0,0.0,1.0,3.0,19.0,17,10,15,13,9


In [70]:
seller_info.shape

(5909925, 10)

In [71]:
data.shape

(5909925, 20)

In [72]:
user_seller_info.shape

(101185, 19)

In [73]:
# for i in range(10):
#     data = pd.merge(data,seller_info.iloc[:,i],how='left',on=["seller_id"])
# data.head()
data = data.merge(user_seller_info,how="left",on=["seller_id","user_id"])

In [74]:
data.head()

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,train_test,age_range,gender,...,brand_number_0,item_number_1,cat_number_1,brand_number_1,item_number_2,cat_number_2,brand_number_2,item_number_3,cat_number_3,brand_number_3
0,66912,203092,1095,3125,7118.0,1110,0,train,5.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,66632,461173,737,4564,1803.0,1103,0,train,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,122117,1018773,737,82,2971.0,807,0,train,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,173592,383058,1113,1358,966.0,604,0,train,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,381740,927745,302,3546,3939.0,1013,0,train,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
data['brand_id'] = data['brand_id'].astype('int32') 
data['age_range'] = data['age_range'].astype('int32') 
data['gender'] = data['gender'].astype('int32') 

NameError: name 'df' is not defined

In [78]:
data.shape

(5909925, 37)

In [77]:
data.isnull().sum()

user_id                       0
item_id                       0
cat_id                        0
seller_id                     0
brand_id                      0
time_stamp                    0
action_type                   0
train_test                    0
age_range                     0
gender                        0
user_action_0                 0
user_action_1                 0
user_action_2                 0
user_action_3                 0
total_action_x                0
item_num                      0
cat_num                       0
seller_num                    0
brand_num                     0
time_num                      0
user_seller_action_0    5731706
user_seller_action_1    5731706
user_seller_action_2    5731706
user_seller_action_3    5731706
total_action_y          5731706
item_number_0           5731706
cat_number_0            5731706
brand_number_0          5731706
item_number_1           5731706
cat_number_1            5731706
brand_number_1          5731706
item_num

In [75]:
data = data.dropna(axis=0,how='any')# 去除含nan的行

MemoryError: Unable to allocate 1.10 GiB for an array with shape (25, 5909925) and data type float64

In [59]:
data.shape

(534595, 48)

In [60]:
seller_info.duplicated()

0           False
1           False
2           False
3           False
4           False
            ...  
14144946     True
14144947     True
14144948     True
14144949     True
14144950     True
Length: 14144951, dtype: bool

In [61]:
seller_info = seller_info.drop_duplicates()

In [62]:
seller_info.shape

(4995, 10)

In [63]:
data = data.merge(seller_info,how="left",on=["seller_id"])

In [64]:
print(data.head())

   user_id  item_id  cat_id  seller_id  brand_id  time_stamp  action_type  \
0   127994   470444     748       1102      1214        1031            0   
1   246788   732624    1181       4729       843        1106            3   
2   216253   476470    1095       4219      6887        1106            0   
3   219676   259310    1467       3578      3091        1101            0   
4   384784   533925    1553       3828      1446        1025            3   

   age_range  gender  age_0.0  ...  brand_number_3  seller_action_0  \
0          0       0        1  ...             0.0           147988   
1          5       0        0  ...             1.0             2983   
2          3       0        0  ...             1.0             1873   
3          6       0        0  ...             1.0            19643   
4          2       0        0  ...             1.0           205935   

   seller_action_1  seller_action_2  seller_action_3  total_action_seller  \
0              121             70

In [65]:
data = data.drop_duplicates()

In [66]:
train = pd.read_csv(r"C:\Users\Superstar\Desktop\data_format1\data_format1\train_format1.csv").drop_duplicates()
train.head()

,user_id,merchant_id,label
0,34176,3906,0
1,34176,121,0
2,34176,4356,1
3,34176,2217,0
4,230784,4818,0


In [67]:
train.rename(columns={'merchant_id':'seller_id'},inplace=True)

In [68]:
data = pd.merge(data,train,how='left',on=['user_id','seller_id'])

In [69]:
data.head()

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,age_range,gender,age_0.0,...,seller_action_0,seller_action_1,seller_action_2,seller_action_3,total_action_seller,item_num_seller,cat_num_seller,brand_num_seller,time_num_seller,label
0,127994,470444,748,1102,1214,1031,0,0,0,1,...,147988,121,7064,13522,169797,3426,37,2,185,0
1,246788,732624,1181,4729,843,1106,3,5,0,0,...,2983,8,146,177,8043,495,33,2,160,0
2,216253,476470,1095,4219,6887,1106,0,3,0,0,...,1873,7,211,168,6478,262,8,2,167,0
3,219676,259310,1467,3578,3091,1101,0,6,0,0,...,19643,51,2637,1649,27558,601,6,2,182,1
4,384784,533925,1553,3828,1446,1025,3,2,0,0,...,205935,422,9887,16325,236397,4209,53,2,185,0


In [70]:
data.shape

(534595, 58)

In [71]:
data.isnull().sum()

user_id                 0
item_id                 0
cat_id                  0
seller_id               0
brand_id                0
time_stamp              0
action_type             0
age_range               0
gender                  0
age_0.0                 0
age_1.0                 0
age_2.0                 0
age_3.0                 0
age_4.0                 0
age_5.0                 0
age_6.0                 0
age_7.0                 0
age_8.0                 0
female                  0
male                    0
unknown                 0
user_action_0           0
user_action_1           0
user_action_2           0
user_action_3           0
total_action_x          0
item_num                0
cat_num                 0
seller_num              0
brand_num               0
time_num                0
user_seller_action_0    0
user_seller_action_1    0
user_seller_action_2    0
user_seller_action_3    0
total_action_y          0
item_number_0           0
cat_number_0            0
brand_number

In [72]:
import numpy as np
import xgboost as xgb
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score


In [73]:
x = data.loc[:,data.columns != 'label']
y = data.loc[:,data.columns == 'label']
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.15, random_state = 2021)


In [74]:
del x_train['user_id'],x_train['seller_id'],x_test['user_id'],x_test['seller_id']

In [75]:
x_test

,item_id,cat_id,brand_id,time_stamp,action_type,age_range,gender,age_0.0,age_1.0,age_2.0,...,brand_number_3,seller_action_0,seller_action_1,seller_action_2,seller_action_3,total_action_seller,item_num_seller,cat_num_seller,brand_num_seller,time_num_seller
144316,610288,1130,2014,1111,0,5,0,0,0,0,...,0.0,6820,10,617,335,12262,260,9,2,182
274838,510392,1271,6987,1111,2,0,0,1,0,0,...,0.0,1619,2,97,122,3383,322,8,9,178
308625,553331,800,5313,1111,2,2,0,0,0,1,...,1.0,1471,6,597,244,3047,24,8,2,166
297307,105272,154,6143,920,3,3,0,0,0,0,...,1.0,70526,190,4992,6738,87422,701,21,2,185
409670,843822,737,3894,1110,0,3,0,0,0,0,...,0.0,1274,0,131,180,3585,97,1,1,181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125401,246167,1606,3776,1106,0,0,0,1,0,0,...,1.0,13425,62,1800,1028,18984,202,30,2,182
214238,483298,656,801,1110,0,4,0,0,0,0,...,0.0,24139,28,1321,1838,27763,885,9,3,185
123262,599466,649,1847,1106,0,0,0,1,0,0,...,0.0,13032,37,2891,1259,20173,158,33,7,184
267779,772645,1368,7622,1103,0,2,0,0,0,1,...,0.0,12057,22,2896,1279,20859,201,27,2,185


In [76]:
def xgb_train(x_train,y_train,x_test,y_test,verbose=True):
#      Min_child_weight = [100,150,200,250,300,350,400,450,500]
#      Colsample_bytree = [0.6,0.65,0.7,0.75,0.8,0.85,0.9]
#      Eta = [0.1,0.15,0.2,0.25,0.3,0.35,0.4]
#      best_score=0
#      best_child = 0
#      best_col = 0
#      best_eta = 0
#      for i in Min_child_weight:
#          for j in Colsample_bytree:
#              for k in Eta:
    model_xgb = xgb.XGBClassifier(
        max_depth=12,# raw8
        n_estimators=1000,
        min_child_weight=200,#400
        colsample_bytree=0.75,#0.84
        subsample=0.8,
        eta=0.1,#0.3
        tree_method='gpu_hist',
        seed=2021
        #colsample_bylevel=i,
        #colsample_bynode=i
    )

    model_xgb.fit(
        x_train,
        y_train,
        eval_metric='auc',
        eval_set=[(x_train,y_train),(x_test,y_test)],
        verbose=verbose,
        early_stopping_rounds=15#早停法，如果auc在10epoch没有进步就stop

    )
    # print("min_child_weight:",i," colsample_bytree:",j," eta:",k)
    print(model_xgb.best_score)
                # if model_xgb.best_score > best_score:
                #     best_score = model_xgb.best_score
                #     best_child = i
                #     best_col = j
                #     best_eta = k
    return model_xgb
xgb_train(x_train,y_train,x_test,y_test)

C:\Users\Superstar\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Superstar\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[0]	validation_0-auc:0.71322	validation_1-auc:0.70959
[1]	validation_0-auc:0.73328	validation_1-auc:0.72542
[2]	validation_0-auc:0.73823	validation_1-auc:0.72859
[3]	validation_0-auc:0.74401	validation_1-auc:0.73305
[4]	validation_0-auc:0.74674	validation_1-auc:0.73576
[5]	validation_0-auc:0.74829	validation_1-auc:0.73736
[6]	validation_0-auc:0.75035	validation_1-auc:0.73958
[7]	validation_0-auc:0.75226	validation_1-auc:0.74128
[8]	validation_0-auc:0.75403	validation_1-auc:0.74267
[9]	validation_0-auc:0.75496	validation_1-auc:0.74406
[10]	validation_0-auc:0.75650	validation_1-auc:0.74557
[11]	validation_0-auc:0.75721	validation_1-auc:0.74674
[12]	validation_0-auc:0.75864	validation_1-auc:0.74765
[13]	validation_0-auc:0.75976	validation_1-auc:0.74849
[14]	validation_0-auc:0.76063	validation_1-auc:0.74909
[15]	validation_0-auc:0.76051	validation_1-auc:0.74902
[16]	validation_0-auc:0.76120	validation_1-auc:0.74944
[17]	validation_0-auc:0.76188	validation_1-auc:0.74995
[18]	validation_0-au

[148]	validation_0-auc:0.83519	validation_1-auc:0.81483
[149]	validation_0-auc:0.83549	validation_1-auc:0.81510
[150]	validation_0-auc:0.83589	validation_1-auc:0.81546
[151]	validation_0-auc:0.83620	validation_1-auc:0.81578
[152]	validation_0-auc:0.83660	validation_1-auc:0.81610
[153]	validation_0-auc:0.83697	validation_1-auc:0.81640
[154]	validation_0-auc:0.83716	validation_1-auc:0.81652
[155]	validation_0-auc:0.83739	validation_1-auc:0.81665
[156]	validation_0-auc:0.83776	validation_1-auc:0.81704
[157]	validation_0-auc:0.83789	validation_1-auc:0.81720
[158]	validation_0-auc:0.83795	validation_1-auc:0.81725
[159]	validation_0-auc:0.83818	validation_1-auc:0.81735
[160]	validation_0-auc:0.83839	validation_1-auc:0.81754
[161]	validation_0-auc:0.83872	validation_1-auc:0.81779
[162]	validation_0-auc:0.83915	validation_1-auc:0.81814
[163]	validation_0-auc:0.83939	validation_1-auc:0.81832
[164]	validation_0-auc:0.83955	validation_1-auc:0.81846
[165]	validation_0-auc:0.83982	validation_1-auc:

[295]	validation_0-auc:0.86938	validation_1-auc:0.84257
[296]	validation_0-auc:0.86967	validation_1-auc:0.84282
[297]	validation_0-auc:0.86998	validation_1-auc:0.84306
[298]	validation_0-auc:0.87008	validation_1-auc:0.84314
[299]	validation_0-auc:0.87018	validation_1-auc:0.84328
[300]	validation_0-auc:0.87027	validation_1-auc:0.84335
[301]	validation_0-auc:0.87042	validation_1-auc:0.84349
[302]	validation_0-auc:0.87056	validation_1-auc:0.84364
[303]	validation_0-auc:0.87086	validation_1-auc:0.84382
[304]	validation_0-auc:0.87118	validation_1-auc:0.84409
[305]	validation_0-auc:0.87129	validation_1-auc:0.84420
[306]	validation_0-auc:0.87137	validation_1-auc:0.84425
[307]	validation_0-auc:0.87165	validation_1-auc:0.84441
[308]	validation_0-auc:0.87191	validation_1-auc:0.84453
[309]	validation_0-auc:0.87199	validation_1-auc:0.84465
[310]	validation_0-auc:0.87216	validation_1-auc:0.84476
[311]	validation_0-auc:0.87221	validation_1-auc:0.84482
[312]	validation_0-auc:0.87245	validation_1-auc:

[441]	validation_0-auc:0.89231	validation_1-auc:0.86126
[442]	validation_0-auc:0.89242	validation_1-auc:0.86137
[443]	validation_0-auc:0.89247	validation_1-auc:0.86142
[444]	validation_0-auc:0.89256	validation_1-auc:0.86146
[445]	validation_0-auc:0.89267	validation_1-auc:0.86152
[446]	validation_0-auc:0.89280	validation_1-auc:0.86162
[447]	validation_0-auc:0.89297	validation_1-auc:0.86176
[448]	validation_0-auc:0.89314	validation_1-auc:0.86195
[449]	validation_0-auc:0.89321	validation_1-auc:0.86198
[450]	validation_0-auc:0.89325	validation_1-auc:0.86197
[451]	validation_0-auc:0.89343	validation_1-auc:0.86217
[452]	validation_0-auc:0.89352	validation_1-auc:0.86217
[453]	validation_0-auc:0.89367	validation_1-auc:0.86230
[454]	validation_0-auc:0.89376	validation_1-auc:0.86241
[455]	validation_0-auc:0.89383	validation_1-auc:0.86251
[456]	validation_0-auc:0.89396	validation_1-auc:0.86255
[457]	validation_0-auc:0.89419	validation_1-auc:0.86271
[458]	validation_0-auc:0.89420	validation_1-auc:

[588]	validation_0-auc:0.90897	validation_1-auc:0.87419
[589]	validation_0-auc:0.90906	validation_1-auc:0.87432
[590]	validation_0-auc:0.90920	validation_1-auc:0.87448
[591]	validation_0-auc:0.90942	validation_1-auc:0.87467
[592]	validation_0-auc:0.90952	validation_1-auc:0.87478
[593]	validation_0-auc:0.90959	validation_1-auc:0.87483
[594]	validation_0-auc:0.90978	validation_1-auc:0.87498
[595]	validation_0-auc:0.90995	validation_1-auc:0.87517
[596]	validation_0-auc:0.91012	validation_1-auc:0.87525
[597]	validation_0-auc:0.91023	validation_1-auc:0.87535
[598]	validation_0-auc:0.91025	validation_1-auc:0.87530
[599]	validation_0-auc:0.91043	validation_1-auc:0.87546
[600]	validation_0-auc:0.91046	validation_1-auc:0.87548
[601]	validation_0-auc:0.91054	validation_1-auc:0.87554
[602]	validation_0-auc:0.91071	validation_1-auc:0.87575
[603]	validation_0-auc:0.91079	validation_1-auc:0.87576
[604]	validation_0-auc:0.91093	validation_1-auc:0.87587
[605]	validation_0-auc:0.91095	validation_1-auc:

[734]	validation_0-auc:0.92266	validation_1-auc:0.88492
[735]	validation_0-auc:0.92272	validation_1-auc:0.88492
[736]	validation_0-auc:0.92281	validation_1-auc:0.88498
[737]	validation_0-auc:0.92283	validation_1-auc:0.88498
[738]	validation_0-auc:0.92296	validation_1-auc:0.88503
[739]	validation_0-auc:0.92311	validation_1-auc:0.88520
[740]	validation_0-auc:0.92317	validation_1-auc:0.88525
[741]	validation_0-auc:0.92327	validation_1-auc:0.88535
[742]	validation_0-auc:0.92335	validation_1-auc:0.88536
[743]	validation_0-auc:0.92343	validation_1-auc:0.88545
[744]	validation_0-auc:0.92356	validation_1-auc:0.88551
[745]	validation_0-auc:0.92360	validation_1-auc:0.88559
[746]	validation_0-auc:0.92374	validation_1-auc:0.88576
[747]	validation_0-auc:0.92378	validation_1-auc:0.88580
[748]	validation_0-auc:0.92382	validation_1-auc:0.88586
[749]	validation_0-auc:0.92383	validation_1-auc:0.88583
[750]	validation_0-auc:0.92386	validation_1-auc:0.88583
[751]	validation_0-auc:0.92399	validation_1-auc:

[880]	validation_0-auc:0.93305	validation_1-auc:0.89310
[881]	validation_0-auc:0.93314	validation_1-auc:0.89319
[882]	validation_0-auc:0.93327	validation_1-auc:0.89329
[883]	validation_0-auc:0.93341	validation_1-auc:0.89340
[884]	validation_0-auc:0.93342	validation_1-auc:0.89340
[885]	validation_0-auc:0.93347	validation_1-auc:0.89347
[886]	validation_0-auc:0.93352	validation_1-auc:0.89353
[887]	validation_0-auc:0.93362	validation_1-auc:0.89357
[888]	validation_0-auc:0.93367	validation_1-auc:0.89363
[889]	validation_0-auc:0.93370	validation_1-auc:0.89362
[890]	validation_0-auc:0.93376	validation_1-auc:0.89370
[891]	validation_0-auc:0.93378	validation_1-auc:0.89367
[892]	validation_0-auc:0.93388	validation_1-auc:0.89376
[893]	validation_0-auc:0.93392	validation_1-auc:0.89377
[894]	validation_0-auc:0.93397	validation_1-auc:0.89378
[895]	validation_0-auc:0.93403	validation_1-auc:0.89389
[896]	validation_0-auc:0.93408	validation_1-auc:0.89399
[897]	validation_0-auc:0.93411	validation_1-auc:

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.75, eta=0.1, gamma=0,
              gpu_id=0, importance_type='gain', interaction_constraints='',
              learning_rate=0.100000001, max_delta_step=0, max_depth=12,
              min_child_weight=200, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=16, num_parallel_tree=1,
              random_state=2021, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=2021, subsample=0.8, tree_method='gpu_hist',
              validate_parameters=1, verbosity=None)

In [77]:
# pred = pd.read_csv(r"C:\Users\Superstar\Desktop\sample_submission.csv")
# pred.rename(columns={1:'seller_id'},inplace = True)
# pred['prob'] = 

In [78]:
predict

,user_id,seller_id,prob
0,163968,4605,0.5
1,360576,1581,0.5
2,98688,1964,0.5
3,98688,3645,0.5
4,295296,3361,0.5
...,...,...,...
261472,228479,3111,0.5
261473,97919,2341,0.5
261474,97919,3971,0.5
261475,32639,3536,0.5


In [79]:
x

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,age_range,gender,age_0.0,...,brand_number_3,seller_action_0,seller_action_1,seller_action_2,seller_action_3,total_action_seller,item_num_seller,cat_num_seller,brand_num_seller,time_num_seller
0,127994,470444,748,1102,1214,1031,0,0,0,1,...,0.0,147988,121,7064,13522,169797,3426,37,2,185
1,246788,732624,1181,4729,843,1106,3,5,0,0,...,1.0,2983,8,146,177,8043,495,33,2,160
2,216253,476470,1095,4219,6887,1106,0,3,0,0,...,1.0,1873,7,211,168,6478,262,8,2,167
3,219676,259310,1467,3578,3091,1101,0,6,0,0,...,1.0,19643,51,2637,1649,27558,601,6,2,182
4,384784,533925,1553,3828,1446,1025,3,2,0,0,...,1.0,205935,422,9887,16325,236397,4209,53,2,185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534590,120564,272027,662,4044,82,914,0,2,0,0,...,0.0,160313,278,8683,13923,187241,1791,27,2,185
534591,423498,246330,662,3958,4248,1111,0,3,0,0,...,0.0,6332,10,376,563,11239,399,18,2,183
534592,329435,601604,154,1102,1214,1108,0,6,2,0,...,0.0,147988,121,7064,13522,169797,3426,37,2,185
534593,311665,945652,602,2664,8154,1111,2,0,0,1,...,0.0,23220,49,2892,834,29659,670,17,2,184


In [80]:
test.isnull().sum()

user_id        0
item_id        0
cat_id         0
seller_id      0
brand_id       0
time_stamp     0
action_type    0
dtype: int64

In [81]:
test = pd.merge(predict,x,how='left',on=['user_id','seller_id'])
test

,user_id,seller_id,prob,item_id,cat_id,brand_id,time_stamp,action_type,age_range,gender,...,brand_number_3,seller_action_0,seller_action_1,seller_action_2,seller_action_3,total_action_seller,item_num_seller,cat_num_seller,brand_num_seller,time_num_seller
0,163968,4605,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,360576,1581,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,98688,1964,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,98688,3645,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,295296,3361,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261472,228479,3111,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
261473,97919,2341,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
261474,97919,3971,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
261475,32639,3536,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
